In [1]:
from scipy.io.wavfile import read as read_wav
from IPython.display import Audio
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn as skl
import librosa as lr
import numpy as np
import logging
import random
import scipy
import os

In [2]:
np.random.seed(5)
logging.warnings.filterwarnings('ignore')

In [3]:
dataset_root = '..'

### Genre Dictionary:

In [4]:
genres = {
    'disco': 0,
    'country': 1
}

### Hyper Parameters:

In [5]:
hparams = {
    'samplerate': 22050,
    'seq_length': 256,
    'hop_length': 512,
    'fft_window': 2048,
    'num_classes': len(genres)
}

### Extract Features:

In [6]:
def extract_features(filepath, hparams):
    
    audio_data_array = lr.load(filepath, sr=hparams['samplerate'], mono=True)
    
    # normalized = lr.util.normalize(audio_data_array[0])
    
    features = [
        
        # MFCC Features:
        lr.feature.mfcc(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),

        # Chroma STFT:
        lr.feature.chroma_stft(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),
        
        # Spectral Centroid:
        lr.feature.spectral_centroid(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        ),
        
        # Spectral Contrast:
        lr.feature.spectral_contrast(
            y          = audio_data_array[0],
            sr         = hparams['samplerate'],
            hop_length = hparams['hop_length'],
            n_fft      = hparams['fft_window']
        )
    ]
    
    return np.vstack(features)

### Disco Class:

In [7]:
class_1 = [
    'disco'
]

### Country Class:

In [8]:
class_2 = [
    'country'
]

### Allocating Arrays to Store Dataset:

In [9]:
train_data_x = np.zeros(shape=(5 * 140, 40, hparams['seq_length']))
val_data_x = np.zeros(shape=(5 * 40, 40, hparams['seq_length']))
test_data_x = np.zeros(shape=(5 * 20, 40, hparams['seq_length']))

train_data_y = np.zeros(shape=(5 * 140, 2))
val_data_y = np.zeros(shape=(5 * 40, 2))
test_data_y = np.zeros(shape=(5 * 20, 2))

### Train Dataset:

In [10]:
preprocess = True

if preprocess == True:

    train_data_counter = 0
    val_data_counter = 0
    test_data_counter = 0

    for genre in class_1:

        print('Genre:', genre, 'Processed.')
        
        m = hparams['seq_length']

        filelist = os.listdir(dataset_root + '/dataset/' + genre)
        
        for file in filelist[0:70]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                train_data_x[train_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                train_data_y[train_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                train_data_counter += 1
        
        for file in filelist[70:90]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                val_data_x[val_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                val_data_y[val_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                val_data_counter += 1
                
        for file in filelist[90:100]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                test_data_x[test_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                test_data_y[test_data_counter] = tf.keras.utils.to_categorical(0, num_classes=2)
                test_data_counter += 1
        

    for genre in class_2:

        print('Genre:', genre, 'Processed.')
        
        m = hparams['seq_length']

        filelist = os.listdir(dataset_root + '/dataset/' + genre)
        
        for file in filelist[0:70]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                train_data_x[train_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                train_data_y[train_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                train_data_counter += 1
        
        for file in filelist[70:90]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                val_data_x[val_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                val_data_y[val_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                val_data_counter += 1
                
        for file in filelist[90:100]:
            features = extract_features(dataset_root + '/dataset/' + genre + '/' + file, hparams)
            for i in range(5):
                test_data_x[test_data_counter, :] = features[:, (i * m): ((i+1) * m)]
                test_data_y[test_data_counter] = tf.keras.utils.to_categorical(1, num_classes=2)
                test_data_counter += 1
            
    if 'prepared' not in os.listdir('.'):
        os.mkdir('prepared')
    np.save('./prepared/train_data_x.npy', train_data_x, allow_pickle=False)
    np.save('./prepared/train_data_y.npy', train_data_y, allow_pickle=False)
    np.save('./prepared/val_data_x.npy', val_data_x, allow_pickle=False)
    np.save('./prepared/val_data_y.npy', val_data_y, allow_pickle=False)
    np.save('./prepared/test_data_x.npy', test_data_x, allow_pickle=False)
    np.save('./prepared/test_data_y.npy', test_data_y, allow_pickle=False)
else:
    train_data_x = np.load('./prepared/train_data_x.npy', allow_pickle=False)
    train_data_y = np.load('./prepared/train_data_y.npy', allow_pickle=False)
    val_data_x = np.load('./prepared/val_data_x.npy', allow_pickle=False)
    val_data_y = np.load('./prepared/val_data_y.npy', allow_pickle=False)
    test_data_x = np.load('./prepared/test_data_x.npy', allow_pickle=False)
    test_data_y = np.load('./prepared/test_data_y.npy', allow_pickle=False)

Genre: disco Processed.
Genre: country Processed.


In [11]:
train_data_x  = np.moveaxis(train_data_x, 1, 2)
val_data_x    = np.moveaxis(val_data_x,   1, 2)
test_data_x   = np.moveaxis(test_data_x,  1, 2)

### Model Creation and Training:

In [12]:
class GenreClassifierModel(tf.keras.Model):
    
    def __init__(self, l1, l2):
        
        super().__init__()
        
        self.L1 = tf.keras.layers.LSTM(l1, return_sequences=True)
        self.L2 = tf.keras.layers.LSTM(l2, return_sequences=False)
        self.L3 = tf.keras.layers.Dense(units=hparams['num_classes'], activation="softmax")
        
    def call(self, inputs):
        self.x = self.L1(inputs)
        self.x = self.L2(self.x)
        self.x = self.L3(self.x)
        return self.x

### Training Model:

In [13]:
def create_model(config):
    
    l1 = config['L1']
    l2 = config['L2']
    
    # Model Definition:
    model = GenreClassifierModel(l1, l2)
    
    # Model Compilation:
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
model = create_model({
    'L1': 64,
    'L2': 32
})

In [15]:
hist = model.fit(train_data_x, train_data_y, epochs=200)

Epoch 1/200
22/22 [==============================] - 0s 17ms/step - loss: 0.6614 - accuracy: 0.6043
Epoch 2/200
22/22 [==============================] - 0s 16ms/step - loss: 0.5908 - accuracy: 0.6829
Epoch 3/200
22/22 [==============================] - 0s 16ms/step - loss: 0.5512 - accuracy: 0.7257
Epoch 4/200
22/22 [==============================] - 0s 16ms/step - loss: 0.5258 - accuracy: 0.7471
Epoch 5/200
22/22 [==============================] - 0s 16ms/step - loss: 0.5150 - accuracy: 0.7500
Epoch 6/200
22/22 [==============================] - 0s 18ms/step - loss: 0.5479 - accuracy: 0.7371
Epoch 7/200
22/22 [==============================] - 0s 17ms/step - loss: 0.5131 - accuracy: 0.7586
Epoch 8/200
22/22 [==============================] - 0s 17ms/step - loss: 0.5008 - accuracy: 0.7871
Epoch 9/200
22/22 [==============================] - 0s 17ms/step - loss: 0.5913 - accuracy: 0.7000
Epoch 10/200
22/22 [==============================] - 0s 17ms/step - loss: 0.5756 - accuracy: 0.6886

22/22 [==============================] - 0s 16ms/step - loss: 0.2760 - accuracy: 0.8871
Epoch 83/200
22/22 [==============================] - 0s 16ms/step - loss: 0.2255 - accuracy: 0.9086
Epoch 84/200
22/22 [==============================] - 0s 16ms/step - loss: 0.2515 - accuracy: 0.9014
Epoch 85/200
22/22 [==============================] - 0s 16ms/step - loss: 0.2727 - accuracy: 0.8843
Epoch 86/200
22/22 [==============================] - 0s 16ms/step - loss: 0.2161 - accuracy: 0.9129
Epoch 87/200
22/22 [==============================] - 0s 16ms/step - loss: 0.2787 - accuracy: 0.8800
Epoch 88/200
22/22 [==============================] - 0s 16ms/step - loss: 0.2427 - accuracy: 0.9057
Epoch 89/200
22/22 [==============================] - 0s 16ms/step - loss: 0.2341 - accuracy: 0.9043
Epoch 90/200
22/22 [==============================] - 0s 17ms/step - loss: 0.2822 - accuracy: 0.8929
Epoch 91/200
22/22 [==============================] - 0s 17ms/step - loss: 0.2819 - accuracy: 0.8829
Epo

22/22 [==============================] - 0s 17ms/step - loss: 0.0533 - accuracy: 0.9843
Epoch 163/200
22/22 [==============================] - 0s 16ms/step - loss: 0.0956 - accuracy: 0.9614
Epoch 164/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0409 - accuracy: 0.9829
Epoch 165/200
22/22 [==============================] - 0s 20ms/step - loss: 0.0970 - accuracy: 0.9729
Epoch 166/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0730 - accuracy: 0.9757
Epoch 167/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0949 - accuracy: 0.9614
Epoch 168/200
22/22 [==============================] - 0s 19ms/step - loss: 0.0688 - accuracy: 0.9800
Epoch 169/200
22/22 [==============================] - 0s 17ms/step - loss: 0.0557 - accuracy: 0.9814
Epoch 170/200
22/22 [==============================] - 0s 18ms/step - loss: 0.0532 - accuracy: 0.9843
Epoch 171/200
22/22 [==============================] - 0s 18ms/step - loss: 0.0699 - accuracy: 0

In [16]:
model.evaluate(val_data_x, val_data_y)

7/7 [==============================] - 0s 9ms/step - loss: 0.2152 - accuracy: 0.9300


[0.21515874564647675, 0.9300000071525574]

In [17]:
model.save('./models/model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./models/model/assets
